Text Data Unit Project!

Prepare data as having been done in class and homework

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import os
import re

from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifier
 

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

from sklearn import preprocessing
from sklearn.manifold import TSNE

Write a function that will 
-take in a corpus (such as prepared so far) as input, and
-take another parameter of “classifier_algorithm” for the algorithm to use, the parameter settings for the algorithm should be part of the “classifier_algorithm” parameter
-run the classifier with train_test split, i.e., train on the training set and test on the test set
-report the metrics of precision, recall, f1_score, and accuracy

In [2]:
data_dir = "/Users/katielark/Downloads/"
sdg_names = pd.read_csv(data_dir + "sdg_name_definition.csv")

text_file_name = "osdg-community-data-v2024-04-01.csv"
text_df = pd.read_csv(data_dir + text_file_name,sep = "\t",  quotechar='"')
text_df.drop(text_df.columns.values[0],axis = 1, inplace=True)
text_df = text_df.query("agreement > 0.5 and (labels_positive - labels_negative) > 2").reset_index(drop=True)

In [3]:
#I tried to incorporate a highlighting function (constructed by chatgpt) but when I included it in the function, it caused too many errors.
def highlight_top_4_max(s):
    # Flatten the DataFrame to get the 4 largest values
    top_4_values = s.nlargest(4).values
    return ['background-color: yellow' if v in top_4_values else '' for v in s]

In [4]:
#chatgpt helped me turn this into a for-loop
classifiers = [MultinomialNB, RidgeClassifier, MLPClassifier]
vectorizers = [CountVectorizer, TfidfVectorizer]
def evaluate_classifiers(corpus, vectorizers, classifiers, classlabel, gram_settings=[(1, 1), (1, 2), (2, 2)], n=3):
    #empty dataset
    results = []

    #For-loop FOR days
    for classifier in classifiers:
        for vectorizer in vectorizers:
            for gram in gram_settings:
                
                #creating training and testing data
                X_train, X_test, y_train, y_test = train_test_split(corpus, classlabel, test_size=0.33, random_state=7)
    
                #creating vectors
                vectorizer_instance = vectorizer(ngram_range=gram, stop_words="english", min_df=5)
                X_train_count_vectorizer = vectorizer_instance.fit(X_train)
                X_train_count_vector = X_train_count_vectorizer.transform(X_train)
                X_test_count_vector = X_train_count_vectorizer.transform(X_test) 

                #running the classifier
                count_classifier_clf = classifier().fit(X_train_count_vector, y_train)
                y_pred = count_classifier_clf.predict(X_test_count_vector)

                # get the numbers!!
                accuracy = metrics.accuracy_score(y_test, y_pred)
                precision = metrics.precision_score(y_test, y_pred, average='macro')
                recall = metrics.recall_score(y_test, y_pred, average='macro')
                f1 = metrics.f1_score(y_test, y_pred, average='macro')

                # distinctive features
                #feature_names = X_train_count_vectorizer.get_feature_names_out()
                #important_features = []
               # for labelid in classlabel:
                   # top_n = sorted(zip(count_classifier_clf.feature_log_prob_[labelid-1], feature_names), reverse=True)[:n]
                   # important_features.append([f"SDG {labelid} : {feat} ({coef:.6f})" for coef, feat in top_n])

                # fill in the results
                results.append({
                    "Classifier": classifier.__name__,
                    "Vectorizer": vectorizer.__name__,
                    "Gram": f"{gram[0]}-{gram[1]}",
                    "Accuracy": accuracy,
                    "Precision": precision,
                    "Recall": recall,
                    "F1": f1,
                   # "Important Features": "\n".join(["\n".join(features) for features in important_features])
                })

    # convert results
    results_df = pd.DataFrame(results)

    # print results
    print(results_df.to_string(index=False))

In [5]:
corpus = text_df.text
sdg_num = text_df.sdg

In [6]:
evaluate_classifiers(corpus, vectorizers, classifiers, sdg_num, gram_settings=[(1, 1), (1,2), (2,2)], n=3)

     Classifier      Vectorizer Gram  Accuracy  Precision   Recall       F1
  MultinomialNB CountVectorizer  1-1  0.843729   0.826635 0.810185 0.815935
  MultinomialNB CountVectorizer  1-2  0.849212   0.838086 0.807411 0.817072
  MultinomialNB CountVectorizer  2-2  0.767421   0.752991 0.710476 0.722522
  MultinomialNB TfidfVectorizer  1-1  0.780101   0.828430 0.693200 0.709172
  MultinomialNB TfidfVectorizer  1-2  0.769820   0.835783 0.674100 0.690059
  MultinomialNB TfidfVectorizer  2-2  0.733036   0.797466 0.643452 0.664464
RidgeClassifier CountVectorizer  1-1  0.817455   0.802288 0.788290 0.794092
RidgeClassifier CountVectorizer  1-2  0.818483   0.802739 0.790858 0.795912
RidgeClassifier CountVectorizer  2-2  0.682774   0.659856 0.641576 0.648487
RidgeClassifier TfidfVectorizer  1-1  0.879941   0.865620 0.850780 0.856499
RidgeClassifier TfidfVectorizer  1-2  0.887252   0.875051 0.858297 0.864756
RidgeClassifier TfidfVectorizer  2-2  0.776445   0.753899 0.727623 0.737443
  MLPClassif

In [ ]:
#Highest overall: Ridge/Tfidf/Bi, MPL/Tfidf/Bi

Run various classification algorithms on the UN SDG labeled data we have been using in class. 

-multinomial naive bayes (multinomialnb)
-multilayer perceptron (sklearn.neural_network.MLPClassifier, note to use min_df to ensure computation can be completed in reasonable time)
-ridge regression (sklearn.linear_model.Ridge)

Combine with different preprocessing settings
-use count vectors vs. tfidf vectors
-use unigram only, bigram only, or both

Evaluate all performance results in one table, with all configurations, bold face the numbers that give the best performance.
For each SDG, what are the most differentiating features according to the classifier?
Are there any overlaps between the SDG vocabularies?

In [19]:
#using modified function to get important features (available only with multinomialNB)
vectorizers = [CountVectorizer, TfidfVectorizer]
def evaluate_features(corpus, vectorizers, classifier, classlabel, gram_settings=[(1, 1), (1, 2), (2, 2)], n=3):
     for vectorizer in vectorizers:
            for gram in gram_settings:
                #creating training and testing data
                X_train, X_test, y_train, y_test = train_test_split(corpus, sdg_num, test_size=0.33, random_state=7)
    
                #creating vectors
                X_train_count_vectorizer = vectorizer(ngram_range=gram, stop_words="english", min_df=5)
                X_train_count_vectorizer.fit(X_train)
                X_train_count_vector = X_train_count_vectorizer.transform(X_train)
                X_test_count_vector = X_train_count_vectorizer.transform(X_test) 
    
                #running the classifier
                count_classifier_clf = classifier().fit(X_train_count_vector, y_train)
                y_pred = count_classifier_clf.predict(X_test_count_vector)

                #getting all the numbers for the report
                metrics.classification_report(y_test,y_pred)
    
                #printing the results
                print("accuracy = {:.4}".format(metrics.accuracy_score(y_test, y_pred)))
                print("macro-averaged precision = {:.4}".format(metrics.precision_score(y_test, y_pred, average = 'macro')))
                print("macro-averaged recall = {:.4}".format(metrics.recall_score(y_test, y_pred, average = 'macro')))
                print("macro-averaged f1 = {:.4}".format(metrics.f1_score(y_test, y_pred, average = 'macro')))

                #printing most important features
                feature_names = X_train_count_vectorizer.get_feature_names_out()  
                for labelid in classlabel:
                    top_n = sorted(zip(count_classifier_clf.feature_log_prob_[labelid], feature_names), reverse=True)[:n]
                    for coef, feat in top_n:
                        print("SDG {} : {:30}  {:.6f}".format(labelid+1, feat, coef))
                        print("")

In [20]:
corpus = text_df.text
sdg_num = text_df.sdg

In [23]:
evaluate_features(corpus, vectorizers, MultinomialNB, [2,8,10,14], gram_settings=[(1, 1), (1, 2), (2, 2)], n=3)

accuracy = 0.8437
macro-averaged precision = 0.8266
macro-averaged recall = 0.8102
macro-averaged f1 = 0.8159
SDG 3 : health                          -3.656426

SDG 3 : care                            -4.013243

SDG 3 : services                        -5.026972

SDG 9 : development                     -4.910686

SDG 9 : countries                       -4.954048

SDG 9 : infrastructure                  -5.066067

SDG 11 : urban                           -4.486567

SDG 11 : development                     -4.826011

SDG 11 : transport                       -4.966711

SDG 15 : forest                          -4.351785

SDG 15 : biodiversity                    -4.446839

SDG 15 : forests                         -4.903450

accuracy = 0.8492
macro-averaged precision = 0.8381
macro-averaged recall = 0.8074
macro-averaged f1 = 0.8171
SDG 3 : health                          -3.993201

SDG 3 : care                            -4.350017

SDG 3 : services                        -5.363747

SDG 9 : d

Most differentiating features: 
Each label has its own distinct words that relate directly to the topic that is covered in the goal it represents.
In the code above, we pulled out the most informative features of each class (examining classes 3, 9, 11, 15). Each word that appears in the report above is observed most commonly in the associated class. The differences between them are summative of the topic covered in the goal. 

We observe overlaps in the vocabulary among SDG 9 and SDG 11- the classifier determines that the word development is an important feature in both classes. Additionally, it is interesting to observe that many of the words pulled out as important features by the classifier (within the same classlabel) are synonyms or directly related (ie, health, care and health care or forests and forest). 

Take the main text content from these pages, and feed them into your classifier and see how your model classifies them. Are the classifications reasonable? 
Find a case where your classification is not reasonable and explain what the model does that leads to the not ideal classification.
http://gianttortoise.org/en/beyond-tracking
https://www.dhs.gov/blue-campaign/what-human-trafficking
https://www.dol.gov/agencies/odep/program-areas/individuals/older-workers
https://michigantoday.umich.edu/2022/08/26/positively-breaking-the-age-code/

In [66]:
#creating new dataframe with copied text from the websites. Two entries for each website so that "stratify" can work in the function (recommended by chatgpt)
corpus = pd.DataFrame({
    'text': ["""Beyond Tracking
Galapagos Giant Tortoise Feeding Ecology
Having discovered some of the mechanisms governing migration and other movements, we wanted to place our new knowledge into a wider ecological and conservation context. We knew that food availability influenced tortoise migrations, but we had little data on tortoise diets. Previous studies had been conducted in the 1980s before the explosion of introduced species on Santa Cruz Island.

We spent several hundred hours observing tortoises and recording all feeding activity, noting the plant species eaten and other details such as bites per minute. We also studied how plant communities change along the elevation gradient.

Tortoises eat at least 96 different plant species. Young mature leaves are preferred, and fruit also makes up a large part of the diet.

Galapagos tortoises consume many plant species that were introduced to Galapagos by people often preferring these over native and endemic plant species. Many of these species were brought to Galapagos as food sources for people and livestock and are highly nutritious so it is not surprising that tortoises feed on them. We found that the physical condition of tortoises may even be improved when feeding on plant species introduced to Galapagos by people.

An unfortunate consequence of feeding heavily on fruits from non-native plant species introduced to Galapagos, such as guava, is that as tortoises eat the fruit, they are also ingesting the seeds they contain. We found that an average pile of tortoise poo contains several hundred seeds of the highly invasive guava tree. 

We also found that it might take two to three weeks for a seed to pass through a tortoise’s digestive tract and during this time, a migrating tortoise may travel several kilometers.  For this reason, tortoises are capable of dispersing huge numbers of seeds over large distances.  This accelerates the spread of these species, which can be highly invasive and destructive to native Galapagos plant communities.

On the other hand, tortoises are also dispersing the seeds of many native species and potentially maintaining them on the islands. Tortoises can genuinely be called the “Gardeners of the Galapagos.”

This story is written up more fully in the following journal articles:

The Dominance of Introduced Plant Species in the Diets of Migratory Galapagos Tortoises Increases with Elevation on a Human‐Occupied Island
Seed dispersal by Galápagos tortoises
Digesta retention time in the Galápagos tortoise (Chelonoidis nigra)
Plant species dispersed by Galapagos tortoises surf the wave of habitat suitability under anthropogenic climate change
Galapagos Giant Tortoises and Human Interactions
As Galapagos develops economically and the human population rises, it will be increasingly important to understand the dynamics of tortoise-human interactions.

Like migratory species all over the world, long distance migration by Galapagos tortoises means that many tortoises leave the protective security of the Galapagos National Park and enter private farmland in the highlands of Santa Cruz and other inhabited islands. This has the potential to lead to challenges both for farmers and for tortoises – several giant tortoises can destroy a field of newly planted maize, for example. On the other hand, the presence of tortoises is compatible with cattle farming.

Furthermore, the private lands of Galapagos are dedicated to different uses, from arable agriculture, to livestock production, to tourism and urban development, all of which have different implications for tortoise conservation and the relationship between tortoise and people. As Galapagos develops economically and the human population rises, it will be increasingly important to understand the dynamics of tortoise-human interactions.

Additional research started in 2017 involves in-depth studies within farmlands to better understand how tortoise movements and behavior might depend on land use and habitat fragmentation, and which strategies could be implemented together with land owners and stakeholders to solve these potential conflicts and their consequences.

We have integrated research on this issue into the Galapagos Tortoise Movement Ecology Program. First, a brief stakeholder workshop was initiated in 2018 to bring private landowners, researchers, and members of the Galapagos National Park Directorate and local institutions together to generate a constructive dialogue to share experiences and discuss strategies and solutions to potential conflicts between people and tortoises.

Attitudes toward tortoises were almost exclusively positive or benign, with landowners recognizing the importance of tortoises to the entire economy of Galapagos, and the non-trivial recognition that the “tortoises were here first.” However, if tortoises are perceived to limit rather than enhance economic opportunity, this situation could change.

As a follow-up to this work, we facilitated a study on tortoise ecology in private lands conducted by PhD student Kyana Pike. The study involved analysis of how tortoise ranging and behavior in private lands are influenced by land use type and human infrastructure such as roads, fences, and natural and artificial ponds.

Global Positioning System (GPS)-tagged tortoises spend an average of 150 days per year in private lands, and a tortoise uses an average of four farms, up to a maximum of 24 different farms. Use of multiple farms under multiple different land uses by a single tortoise indicates that finding shared solutions to tortoise conservation and conflict mitigation on Santa Cruz island will require cooperation across the agricultural zone.

Contrary to our expectations, fences in their current configuration seem to present few serious barriers to tortoise movements. Most fences are in a poor state of repair, however even fences designed to keep tortoises out of crop areas are relatively porous.

This important research is in its early days and we will be reporting on outputs in detail in the future. Initial scientific publications on this work are published in the following journal articles:

Identifying Shared Strategies and Solutions to the Human–Giant Tortoise Interactions in Santa Cruz, Galapagos: A Nominal Group Technique Application
Migration by Galapagos giant tortoises requires landscape-scale conservation efforts
 
 
Our Story
About Us
Our People
Research Projects
Publications
Make a Difference
Donate
Grant a Wish
Volunteer
Our Supporters
Languages
en_USEnglish
es_ESEspañol
Policies
Privacy Policy
Cookie Policy""","""Our research program began with a single simple question – “Do Galapagos tortoises undergo long distance annual migrations?”

Answering this question led to the development of an integrated research program spanning disciplines of movement ecology, reproductive ecology, One Health, and the social sciences.

It has resulted in 20 peer reviewed publications to date and many more to come. All have strong implications for conservation and management of the Galapagos archipelago and its tortoises.

Our program grew organically, and here we describe our research journey over this first decade of the program. Our research is aimed to contribute to big questions in evolutionary ecology and to provide direct and tangible applications for conservation. 

A decade of continuous research may seem impressive – yet it represents perhaps just five percent of the lifespan of the animals on which it is based. We clearly have a little more work to do.

Movement
Beyond Tracking
Health
Results
and Impact
Our Story
About Us
Our People
Research Projects
Publications
Make a Difference
Donate
Grant a Wish
Volunteer
Our Supporters""" , """What Is Human Trafficking?
Human trafficking involves the use of force, fraud, or coercion to obtain some type of labor or commercial sex act. Every year, millions of men, women, and children are trafficked worldwide – including right here in the United States. It can happen in any community and victims can be any age, race, gender, or nationality. Traffickers might use the following methods to lure victims into trafficking situations:

Violence
Manipulation
False promises of well-paying jobs
Romantic relationships 
Language barriers, fear of their traffickers, and/or fear of law enforcement frequently keep victims from seeking help, making human trafficking a hidden crime.

Traffickers look for people who are easy targets for a variety of reasons, including:

Psychological or emotional vulnerability
Economic hardship
Lack of a social safety net
Natural disasters
Political instability
The trauma caused by the traffickers can be so great that many may not identify themselves as victims or ask for help, even in highly public settings.

Many myths and misconceptions exist. Recognizing key indicators of human trafficking is the first step in identifying victims and can help save a life. Not all indicators listed are present in every human trafficking situation, and the presence or absence of any of the indicators is not necessarily proof of human trafficking.

The safety of the public as well as the victim is important. Do not attempt to confront a suspected trafficker directly or alert a victim to any suspicions. It is up to law enforcement to investigate suspected cases of human trafficking.

Visit the links below to learn more about human trafficking and how you can protect yourself and others.

Forced Labor
Image
Forced Labor Icon
Identify a Victim
Image
Identify a Victim Icon
Myths and Misconceptions
Image
Myths and Misconceptions Icon
Exploitation and How to Protect Yourself
Image
Exploitation and How to Protect Yourself Icon
Explotación y Cómo Protegerse A Sí Mismo
Image
Exploitation and How to Protect Yourself Icon
What is Human Trafficking? Infographic
Image
What is Human Trafficking Infograpghic 
¿Qué es la Trata de Personas?
Image
¿Qué es la Trata de Personas? Icon
Topics
Human Trafficking
Keywords
Blue Campaign Human Trafficking

To report suspected human trafficking to Federal law enforcement:
Image
To report suspected human trafficking, call 1-866-347-2423
1-866-347-2423
Para reportar un posible caso de trata de personas:
Image
Para reportar un posible caso de trata de personas: 1-866-347-2423
1-866-347-2423
To get help from the National Human Trafficking Hotline:
Image
To get help from the National Human Trafficking Hotline, call 1-888-373-7888
1-888-373-7888
or text HELP or INFO to BeFree (233733)

Obtenga ayuda de la Línea Directa Nacional de Trata de Personas:
Image
Obtenga ayuda de la Línea Directa Nacional de Trata de Personas: 1-888-373-7888
1-888-373-7888
o enviando un mensaje de texto con HELP o INFO a BeFree (233733)

Last Updated: 09/22/2022

Was this page helpful?
Yes No
Return to top
About Blue Campaign
What Is Human Trafficking?
How You Can Help
Learning Center
Events and Initiatives
Blue Campaign. One Voice. One Mission. End Human Trafficking.
Facebook
X
Instagram
Email
Contact Blue Campaign
Report Suspected Human Trafficking: 1-866-347-2423
Get Help from the National Human Trafficking Hotline: 1-888-373-7888
U.S. Department of Homeland Security Seal
DHS.gov/Blue-Campaign

An official website of the U.S. Department of Homeland Security

About Blue Campaign
Accessibility
FOIA Requests
Privacy Policy
DHS.gov
""",""" Forced labor occurs when individuals are compelled against their will to provide work or service through the use of force, fraud, or coercion. This crime happens both in the United States and overseas. The International Labour Organization (ILO) estimated that 24.9 million people around the globe were in forced labor as of 2016.  Victims are rarely able to seek help for various reasons. For instance, they may be hindered by language barriers, or they may physically be unable to leave the premises to seek help if their movements are restricted and monitored by their employer.

Close all
          Open all
Who is Affected 
Traffickers who exploit people for forced labor do not discriminate. Neither do employers: Victims can be any age, race, religious affiliation, gender identity, or nationality. They may also come from any socioeconomic group. Certain risk factors, however, may make certain individuals more vulnerable to forced labor than others. These include:

Unstable immigration status
Language barriers
Poverty and lack of basic needs like food, shelter, and safety
The psychological effects of a recent or past trauma
Lack of social support systems like friends, family, and community
Physical or developmental disabilities
Traffickers frequently target vulnerable populations, such as children, individuals without lawful immigration status, those with debts, and those who are isolated, impoverished, or disabled, to name a few. U.S. citizens, foreign nationals, women, men, and children can all be victims of forced labor.

What Does it Look Like?
Forced labor indicators often intersect and overlap, feeding into each other. Some signs may be more subtle than others. For instance, the individual could fall into debt during the recruitment process, compounded by an employer who takes unexpected deductions from their pay. The worker then cannot repay their debt as quickly as they had anticipated, falling further into debt bondage as a result of both deception and withholding of wages. Does an individual appear to be monitored when talking to or interacting with others? Are they living in dangerous, overcrowded, or inhumane lodging provided by an employer? Are they isolated, physically or culturally? A “yes” to any of these questions could indicate a potential forced labor situation.

Indicators of forced labor may take place at any point during the recruitment and employment process. Forced labor could begin during the worker’s recruitment process to force the acceptance of the job, to deceive the worker into an exploitative job, or to create a situation of debt bondage by charging recruitment fees that are virtually impossible for the workers to repay. Once the person is working, an employer may also force, defraud, or coerce the victim to perform work not agreed to at the time of recruitment. A worker may agree to do a job, find the conditions are not what were expected and agreed upon, but be prevented from leaving the job by their employer.

A list of indicators used to identify whether forced labor is occurring can be found on the ILO’s website.

Forced Labor in the United States
Forced Labor in Industry
Goods Produced with Forced Labor Destined for U.S. Markets
How You Can Help
Learn More
Topics
Human Trafficking
Keywords
Blue Campaign Combatting Human Trafficking Human Trafficking Forced Labor

To report suspected human trafficking to Federal law enforcement:
Image
To report suspected human trafficking, call 1-866-347-2423
1-866-347-2423
Para reportar un posible caso de trata de personas:
Image
Para reportar un posible caso de trata de personas: 1-866-347-2423
1-866-347-2423
To get help from the National Human Trafficking Hotline:
Image
To get help from the National Human Trafficking Hotline, call 1-888-373-7888
1-888-373-7888
or text HELP or INFO to BeFree (233733)

Obtenga ayuda de la Línea Directa Nacional de Trata de Personas:
Image
Obtenga ayuda de la Línea Directa Nacional de Trata de Personas: 1-888-373-7888
1-888-373-7888
o enviando un mensaje de texto con HELP o INFO a BeFree (233733)

Last Updated: 04/05/2024

Was this page helpful?
""" , """Today, a confluence of factors is prompting America to change the way it thinks about age and work. The economic downturn, shifting perceptions of retirement, increased workplace flexibility, and the aging of the "baby boom" generation are all contributing to people working longer. Many of these capable, experienced mature workers develop disabilities as they age, or existing disabilities may become more significant. To retain the talents of these valuable, skilled workers, employers can implement a variety of workplace practices, many of which benefit all workers and make good business sense. The following resources provide more information about the topic of older workers:

Reports from ODEP's NTAR Leadership Center
National Technical Assistance and Research Center to Promote Leadership for Increasing the Employment and Economic Independence of Adults with Disabilities (NTAR Leadership Center) issued these reports that examine the disability implications of an aging workforce.

Community College Briefs
Postsecondary education is increasingly important for older job seekers' reemployment. Yet, they may face potential challenges in accessing and completing education and training due to their greater likelihood of having acquired age-related disabilities. The following three briefs provide new research data and findings on older students and dislocated workers researched by the NTAR Leadership Center. Existing data, such as that from the Integrated Postsecondary Education Data System, does not track the numbers of dislocated workers enrolled at community colleges; and nearly three-quarters of community colleges reported very few students with disabilities enrolled (less than three percent of their student population). Moreover, among older students, unidentified disabilities are not documented, which provides challenges for community colleges to document their statistics on older students with disabilities.

Community College Practices that Serve Older Dislocated Workers (PDF) — This brief highlights strategies and findings at five community colleges serving high numbers of dislocated workers and examines how those practices meet the needs of older workers, some of whom may be aging with or into disabilities.
How Are Community Colleges Serving the Needs of Older Students with Disabilities? (PDF) — To examine the issues related to older students with disabilities, this brief documents the research conducted to learn how colleges—in particular, community colleges—can better support the education and training needs of these students.
Working for Adults: State Policies and Community College Practices to Better Serve Adult Learners at Community Colleges During the Great Recession and Beyond (PDF) — This report synthesizes knowledge about how community colleges serve adults. The first section provides background and context on adults at community colleges, while the second section details the methodology used in this research. Other sections describe the findings on the enrollment of adults at community colleges, recent initiatives that have sought to support adults at community colleges, the state policy and college practices related to adults' enrollment, and research on student outcomes and the implications for what is known about state policy and college practice. The final section highlights recommendations for policymakers and practitioners interested in serving adults at community colleges, including those with disabilities.
ODEP and other DOL agency resources
Stay at Work/Return to Work Programs — A resource from the Employer Assistance and Resource Network on Disability Inclusion (EARN) that shows the advantages of stay-at-work and return-to-work (SAW/RTW) programs, discusses successful SAW/RTW retention strategies, and helps employers decide which SAW/RTW strategies will work in their organizations.
Job Accommodation Network guidance on accommodations for employees who are aging.
Accommodation and Compliance Series: Employees who are Aging
Our Aging Workforce: A Look at the Benefits of Job Accommodation — JAN's Consultants' Corner
Senior Community Service Employment Program (SCSEP) — ETA-funded community service and work-based program that provides subsidized training for low-income persons 55 or older who are unemployed and have poor employment prospects.
Age Discrimination in Employment Act of 1967
Age Discrimination in Employment Act of 1975
DOL Employment & Training Administration's Older Worker Initiative — The aging and retirement of the baby boom generation will have impacts on many aspects of our society, including possible labor and skill shortages. This initiative looks at ways to encourage older employees to continue working.
Retaining Older Workers — Information from EARN on strategies to retain the talents of older workers, who may develop disabilities as they age, and how to attract new, older workers.
Other resources
Making Work More Flexible: Opportunities and Evidence (PDF) — This report considers the availability, utilization, and demand for workplace flexibility, with a particular emphasis on older workers. Although many aspects of flexibility can benefit workers of any age, the desire of some older workers to phase into retirement introduces some special considerations.
Phased Retirement and Flexible Retirement Arrangements: Strategies for Retaining Skilled Workers (PDF) — Implementing appealing work arrangements that attract and retain workers 50+ may become increasingly important in an organization's bid to survive in today's marketplace. Phased retirement, which allows the employee to reduce work time in his or her current job, is regarded as one strategy to encourage hard-to-replace, experienced workers to postpone leaving the labor force. This report discusses the factors influencing the business need for phased retirement, how to create a phased retirement program, how to market a phased retirement program to employees, challenges in implementing phased retirement, proposed regulatory solutions, and cutting-edge employee programs.
Protecting Family Caregivers from Employment Discrimination (AARP Public Policy Institute) (PDF) — This report is the first in a series of AARP Public Policy Institute papers on issues of eldercare and the workplace. It highlights the realities of changing demographics and issues affecting working caregivers of older adults. It defines family responsibilities discrimination (FRD), explains why FRD is a policy matter, and describes the types of workplace discrimination encountered by working caregivers.
Highlights of a GAO Forum: Engaging and Retaining Older Workers (PDF)
Older Workers: Some Best Practices and Strategies for Engaging and Retaining Older Workers (PDF)
Building Your Career After 50 — AARP resources to assist 50 and older workers looking to switch careers or stay in their profession.
Older Workers: An exploration of the Benefits, Barriers, and Adaptations for Older People in the Workforce — A study from the National Institutes of Health which looks at the experiences and perceptions of paid workers aged 60 years and older. The study explains why older people continue to work and the barriers and facilitators they encounter.
Scroll to Top
Program Areas
State Policy
Research and Evaluation
Initiatives
News and Publications
NDEAM
ADA
About
United States Department of Labor
Office of Disability Employment Policy
An agency within the U.S. Department of Labor

200 Constitution Ave NW
Washington, DC 20210
1-866-4-USA-DOL

1-866-487-2365
Federal Government
White House
Coronavirus Resources
Disaster Recovery Assistance
DisasterAssistance.gov
USA.gov
Notification of EEO Violations
No Fear Act Data
U.S. Office of Special Counsel
Labor Department
About DOL
Guidance Search
Español
Office of Inspector General
Subscribe to the DOL Newsletter
Read the DOL Newsletter
Emergency Accountability Status Link
A to Z Index
About The Site
Freedom of Information Act
Privacy & Security Statement
Disclaimers
Important Website Notices
Plug-Ins Used on DOL.gov
Accessibility Statement""", """Today, a confluence of factors is prompting America to change the way it thinks about age and work. The economic downturn, shifting perceptions of retirement, increased workplace flexibility, and the aging of the "baby boom" generation are all contributing to people working longer. Many of these capable, experienced mature workers develop disabilities as they age, or existing disabilities may become more significant. To retain the talents of these valuable, skilled workers, employers can implement a variety of workplace practices, many of which benefit all workers and make good business sense. The following resources provide more information about the topic of older workers:

Reports from ODEP's NTAR Leadership Center
National Technical Assistance and Research Center to Promote Leadership for Increasing the Employment and Economic Independence of Adults with Disabilities (NTAR Leadership Center) issued these reports that examine the disability implications of an aging workforce.

Community College Briefs
Postsecondary education is increasingly important for older job seekers' reemployment. Yet, they may face potential challenges in accessing and completing education and training due to their greater likelihood of having acquired age-related disabilities. The following three briefs provide new research data and findings on older students and dislocated workers researched by the NTAR Leadership Center. Existing data, such as that from the Integrated Postsecondary Education Data System, does not track the numbers of dislocated workers enrolled at community colleges; and nearly three-quarters of community colleges reported very few students with disabilities enrolled (less than three percent of their student population). Moreover, among older students, unidentified disabilities are not documented, which provides challenges for community colleges to document their statistics on older students with disabilities.

Community College Practices that Serve Older Dislocated Workers (PDF) — This brief highlights strategies and findings at five community colleges serving high numbers of dislocated workers and examines how those practices meet the needs of older workers, some of whom may be aging with or into disabilities.
How Are Community Colleges Serving the Needs of Older Students with Disabilities? (PDF) — To examine the issues related to older students with disabilities, this brief documents the research conducted to learn how colleges—in particular, community colleges—can better support the education and training needs of these students.
Working for Adults: State Policies and Community College Practices to Better Serve Adult Learners at Community Colleges During the Great Recession and Beyond (PDF) — This report synthesizes knowledge about how community colleges serve adults. The first section provides background and context on adults at community colleges, while the second section details the methodology used in this research. Other sections describe the findings on the enrollment of adults at community colleges, recent initiatives that have sought to support adults at community colleges, the state policy and college practices related to adults' enrollment, and research on student outcomes and the implications for what is known about state policy and college practice. The final section highlights recommendations for policymakers and practitioners interested in serving adults at community colleges, including those with disabilities.
ODEP and other DOL agency resources
Stay at Work/Return to Work Programs — A resource from the Employer Assistance and Resource Network on Disability Inclusion (EARN) that shows the advantages of stay-at-work and return-to-work (SAW/RTW) programs, discusses successful SAW/RTW retention strategies, and helps employers decide which SAW/RTW strategies will work in their organizations.
Job Accommodati""","""Age is just a number
Becca Levy, BA '87
Scientist/author Becca Levy, BA ’87, is a professor of both epidemiology and psychology at Yale. (Image courtesy of Levy.)

American author and activist Betty Friedan once observed: “Aging is not ‘lost youth’ but a new stage of opportunity and strength.”

Those who share her optimism may find the pathway to longevity.

Scientist Becca Levy, BA ’87, a leading expert on the psychology of successful aging, says taking an upbeat attitude toward aging can not only improve your physical and mental health as you grow older ― but also may add nearly eight years to your lifespan.

She explains how our positive and negative age beliefs shape our behaviors, health, and, ultimately, our longevity in her new book Breaking the Age Code (William Morrow; April 12, 2022).

Levy also reveals that some health issues commonly associated with old age ― hearing loss, high blood pressure, and cardiovascular disease ― are the products of negative stereotypes and prejudices absorbed from our social surroundings. All too often, these fatalistic attitudes about the inevitability of declining health in later life become self-fulfilling prophecies.

“Age beliefs impact our health in ways big and small,” says Levy, who became interested in psychology as a U-M undergraduate. She completed her graduate work at Harvard, and is now a professor of epidemiology and of psychology at Yale.

“People who have more-positive beliefs tend to show benefits in health outcomes and healing compared to those who hold more-negative ones,” she says.

Surprise!
Over the past 20 years, Levy has conducted groundbreaking studies on different health conditions affected by attitudes toward aging. Her results show some surprising results:

Cognition ― People who stress positive age beliefs enjoy better memory performance.
Physical health ― Patients with favorable attitudes about aging are more likely to recover from severe disability.
Mental health ― Individuals who see aging as a positive experience have lower stress levels.
Longevity ― Younger people who adopt a positive outlook on aging live an average of 7.5 years longer.
While Levy’s research underscores the value of celebrating our advancing years as a time for creativity, exploration, and accomplishment, today’s reality is often quite different. All too frequently, personal views, cultural stereotypes, and institutional biases about aging are tilted in a negative direction.

In American society, old age is presented as something to be feared and avoided. Aging individuals are portrayed as fragile, forgetful, and a burden on society. The elderly are marginalized, ignored, and “put out to pasture.”

Levy describes this pervasive ageism in the U.S. and other countries as the “Silent Epidemic” because it operates, undetected, in so many different realms ― social media, advertising, pop culture, Hollywood, and health care.

Ageism as big business
Try the ABC Method to Bolster Positive Age Beliefs
Awareness:

Jot down five words or phrases that come to mind when you think of an older person.
Create a portfolio of positive older role models you admire.
Notice age beliefs in the media.
Think about ways to increase your intergenerational contacts and interactions.
Blame-shifting:

Find the real cause of unpleasant events or challenges, such as momentary forgetfulness.
Name the company or institution that benefits from negative age stereotypes.
Identify ageism when older workers are targeted.
Challenge:

Present accurate information to debunk negative age stereotypes.
Get involved in politics.
Confront ageism and negative age images in print, television, social media, and advertising.
During America’s early history, views of aging were generally positive. However, in the mid-1800s, this positivity began to wane, giving way to less-favorable age beliefs that have taken hold over time.

“The increase in negativity is due in part to the rise of advertisement and the growth of the antiaging industry,” says Levy, who has studied American age beliefs in written language spanning the past two centuries. “Companies have made a lot of money promoting negative images of aging as a way to sell their anti-aging products.”

The research firm Statista projects the global anti-aging market will generate more than $67 billion in 2022 by peddling pills, creams, tinctures, elixirs, hormonal supplements, and procedures that falsely claim to halt or even reverse aging.

The “medical disability” complex
Another source of ageism has been the increased “medicalization” of aging, Levy says.

“Advertisements typically present images of older people as patients and recipients of medical care,” she says. “These ads are not balanced by images showing the heterogeneity of older adults who come from diverse backgrounds and are engaged in different activities, such as work, volunteering, sports, and recreation.”

Television, movies, and social media have amplified misbeliefs about aging. Ageism has become “click bait” on Facebook, Twitter, Instagram, and YouTube. Agelining has relegated elderly residents to age-segregated senior housing where they are, in effect, “quarantined” from society.

Costly mistake
Ageism in the workplace has cost many senior workers with years of valuable experience in their jobs, livelihoods, and feelings of self-worth. Two-thirds of workers in America said they have witnessed or personally experienced age discrimination in their place of work, according to an AARP survey.

Western medicine relies heavily on negative age stereotypes, with their narrative of inevitable decline, because it’s profitable, according to Levy.

Levy teamed up with an economist and statistician to put a price tag on the health costs resulting from ageism. They found it totaled $63 billion per year in the U.S., more than the cost of morbid obesity.“The multibillion-dollar ‘medical disability complex’ is based on expensive procedures, devices, and pharmaceutical drugs, which are more profitable than prevention efforts,” she says. “When ageism is ignored, doctors are apt to dismiss treatable conditions, such as back pain or depression, as standard features of old age.”
This view of aging as a pathology can result in the undertreatment of elderly patients, she says.

Levy teamed up with an economist and statistician to put a price tag on the health costs resulting from ageism. They found it totaled $63 billion per year in the U.S.,  more than the cost of morbid obesity, one of America’s most expensive chronic conditions.

“The World Health Organization has called ageism the most prevalent and socially acceptable form of prejudice and discrimination today,” says Levy, who has testified before the U.S. Senate on the adverse effects of ageism. She also has contributed to briefs submitted to the U.S. Supreme Court in age-discrimination cases and has participated in United Nations discussions of ageism.

Dislodging the stereotypes
Despite the entrenched negative age stereotypes that permeate American society, Levy is convinced they can be dislodged and replaced by more-favorable views of aging and older people.

“The most important takeaway from my research is that we know age beliefs are malleable,” Levy says. “We can increase our awareness of them, challenge some negative ones, and strengthen some positive ones.”

She demonstrated in one lab study that exposing older participants to subliminal positive messages about aging improved their physical function, including walking and balance. Another study showed that positive age beliefs lowered stress and helped at-risk people ward off the symptoms of Alzheimer’s disease.

Igniting an age liberation movement
Breaking the age code bookcover
(William Morrow, April 2022.)

The time has come to shift from an age-declining to an age-thriving mindset, according to Levy, who presents a blueprint for overcoming structural ageism in her book.

“I think we are getting closer to a tipping point where an age liberation movement will take hold,” she says. “Growing numbers of people are becoming aware and angry about ageism.”

National organizations, such as the American Psychological Association, the Gerontological Society of America, and HelpAge International, have begun issuing urgent warnings about the hazards of ageism. The Gray Panthers in New York City continue to confront age stereotypes and discrimination head-on. In addition, the World Health Organization recently launched its Campaign to Combat Ageism, which 194 countries have endorsed. Levy is serving as a scientific adviser to the campaign.

However, until the groundswell of opposition to structural ageism in our social and policy institutions gains traction, ordinary people will need tools to navigate, question, and challenge negative stereotypes and attitudes on aging.

Levy has developed an “ABC Method” that individuals can use to harness the power of their own positive age beliefs to improve their health. This approach consists of three stages:

Increasing Awareness of negative age beliefs within and around us
Placing Blame on ageism and its societal sources
Challenging negative age beliefs
“As individuals acquire a greater sense of their value as older persons, they are more likely to participate in an age liberation movement,” Levy says. “The movement, in turn, is bound to further increase their sense of value as older persons. This cultural redefinition will contribute to a virtuous cycle.”

Comments
Thomas Cislo - 1972, 1975
Excellent book review and an important topic! Thank you.

Reply

Cliff Douglas - 1980, 1983
This is terrific work and a very important contribution to the public health and welfare of many millions of people. If of interest, I’ve tweeted it out at https://twitter.com/cdoug/status/1563525118562775044?s=21&t=kc0hIo9p9oXKNjYd3eIjJw to let more people know about it. Thanks!

Reply

Thomas F. Higby - 1958 M.D.
I’m frequently asked “How are you doing?”, my reply is: “Never having been at this stage of life before, I don’t know what to expect”. Actually, I think it’s “pretty well”.
I’ll be 90 in December. Today I have been mowing my grass; walk behind mower that I call my self propelled walker. I will have a practice session on my horn; I play in The Livingston County Concert Band. I’m regularly dating with a near 90 woman friend. I’m in contact with medical and H.S class-mates ( a rapidly disappearing bunch) It sounds like a good life. I do not expect anything beyond the grave.
Tom Higby

Reply

Paul Krueger - Years since graduation? 90 years old, 2 degrees UM, 1 Harvard
All sounds too negative.
Take a more positive approach.

Reply

Richard DeVries - 1973
Good article. It’s interesting, however, that the University of Michigan itself is very quick to treat patients differentially based solely on age. Patients over a certain age are quickly transported out of UM to a geriatric floor of St Joe’s with what I have observed as a lower level of care. The attitude of staff, especially younger staff, can be very ageist. I hope individuals in the UM system review this article.

Reply

Kerstin Prof.Dr. Schaper-Lang - 1976 and 1991
Thanks for this wonderful article. It gives hope to all of us! Yes, rethink age: we are the most expirienced and valuable workforce, what an enormous value for any society. Everybody has different genes, so overcome stereotypes.

Reply

Leave a comment:
First Name (required):
Last Name (required):
Email (required, will not be published):
Year(s) of graduation:
Comment:
""", """ CLAUDIA R. CAPOS has more than 25 years of experience in journalism, publications, and media relations. A graduate of the University of Michigan’s School of Journalism, she co-founded and co-published About Ann Arbor Magazine for five years before joining The Detroit News where, over a 10-year period, she held a variety of writing and editing positions and received three Pulitzer Prize nominations. Since leaving The News, Capos has provided communications services to clients ranging from the Big Three automotive manufacturers to small and mid-size companies. She's published articles on business, real estate, and travel in national and regional magazines, as well as major metropolitan newspapers."""],
    'label': [1, 1, 2, 2, 3, 3, 4, 4]
})

In [68]:
print(corpus.head())  
print(corpus.columns)

                                                text  label
0  Beyond Tracking\nGalapagos Giant Tortoise Feed...      1
1  Our research program began with a single simpl...      1
2  What Is Human Trafficking?\nHuman trafficking ...      2
3   Forced labor occurs when individuals are comp...      2
4  Today, a confluence of factors is prompting Am...      3
Index(['text', 'label'], dtype='object')


In [78]:
vectorizers = [CountVectorizer, TfidfVectorizer]
def evaluate_web_features(corpus, vectorizers, classifier, classlabel, gram_settings=[(1, 1), (1, 2), (2, 2)], n=3):
    # Extract text and labels from corpus
    X = corpus['text']
    y = corpus['label']

    for vectorizer in vectorizers:
        for gram in gram_settings:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=7, stratify=y)
            
            vectorizer_instance = vectorizer(ngram_range=gram, stop_words="english")
            X_train_vector = vectorizer_instance.fit_transform(X_train)
            X_test_vector = vectorizer_instance.transform(X_test)
    
          
            clf = classifier().fit(X_train_vector, y_train)
            y_pred = clf.predict(X_test_vector)

            
            print(f"Metrics for {vectorizer.__name__} with ngram_range={gram}:")

            # Printing most important features if the classifier supports it
            feature_names = vectorizer_instance.get_feature_names_out()
            for labelid, label in enumerate(classlabel):
                top_n = sorted(zip(clf.feature_log_prob_[labelid], feature_names), reverse=True)[:n]
                for coef, feat in top_n:
                    print(f" {labelid+1} : {feat:30}  {coef:.6f}")
                    print("\n" + "-"*50 + "\n")

In [80]:
evaluate_web_features(corpus, vectorizers, MultinomialNB, classlabel=[1,2,3,4],gram_settings=[(1,2)], n=7)

Metrics for CountVectorizer with ngram_range=(1, 2):
 1 : tortoises                       -4.879992

--------------------------------------------------

 1 : galapagos                       -5.084787

--------------------------------------------------

 1 : tortoise                        -5.131307

--------------------------------------------------

 1 : species                         -5.536772

--------------------------------------------------

 1 : plant                           -5.777934

--------------------------------------------------

 1 : plant species                   -5.978604

--------------------------------------------------

 1 : human                           -5.978604

--------------------------------------------------

 2 : labor                           -5.358471

--------------------------------------------------

 2 : forced labor                    -5.358471

--------------------------------------------------

 2 : forced                          -5.358471


For the most part, the classifications of the websites above are reasonable. The tortoise tracking website returned words about tortoises and plants, the human trafficking website returned forced labor, help, human trafficking and the website about working in older age returned words related to working, community college and disabilities. 
However, the classification of the final website is inaccurate. It returned features such as journalism, capos, years experience, etc. The website itself is about older age as well. There are two things that I believe contribute to this unreasonable classification. The first is that I copied the author's biography into the second label for website 4. The words returned by the classifier are definintely similar to those in the biography. Additionally, its similarity in content to website 3 caused the classifier to identify unique words between the two. Since website 3 did not have an author's bio, the classifier latched onto those types of words as the distinguishing features. 